In [1]:
import asyncio
from hfc.fabric import Client
import nest_asyncio
nest_asyncio.apply()

In [2]:
loop = asyncio.get_event_loop()
cli = Client(net_profile="test/fixtures/network.json")
org1_admin = cli.get_user('org1.example.com', 'Admin')

Init client with profile=test/fixtures/network.json
create org with name=orderer.example.com
create org with name=org1.example.com
create org with name=org2.example.com
create ca with name=ca-org1
create ca with name=ca-org2
Import orderers = dict_keys(['orderer.example.com'])
Import peers = dict_keys(['peer0.org1.example.com', 'peer1.org1.example.com', 'peer0.org2.example.com', 'peer1.org2.example.com'])


In [3]:
# Query Peer installed chaincodes, make sure the chaincode is installed
response = loop.run_until_complete(cli.query_installed_chaincodes(
               requestor=org1_admin,
               peers=['peer0.org1.example.com'],
               decode=True
               ))

print(response)

cc name atcc, version v1.0, path github.com/atcc


[chaincodes {
  name: "atcc"
  version: "v1.0"
  path: "github.com/atcc"
  id: "\345\255\346\035\244E\003(\r\355\276\376:,.0\224\n\276X\002\357I\253DN\220\242\303\300C\031"
}
]


In [4]:
# Query Peer Joined channel
response = loop.run_until_complete(cli.query_channels(
               requestor=org1_admin,
               peers=['peer0.org1.example.com'],
               decode=True
               ))

response

channel id assetchannel


channels {
  channel_id: "assetchannel"
}

In [5]:
cli.new_channel('assetchannel')

New channel with name = assetchannel


In [6]:
org1_admin = cli.get_user('org1.example.com', 'Admin')

In [7]:
# first get the hash by calling 'query_info'
response = loop.run_until_complete(cli.query_info(
               requestor=org1_admin,
               channel_name='assetchannel',
               peers=['peer0.org1.example.com'],
               decode=True
               ))
response

response status 200


height: 5
currentBlockHash: "\237.\217NQ|)\232\240<\013\250\276g\273\0377\021\030\205b\037\243\203K%\276\r\016\344\341\220"
previousBlockHash: "\215\273\233\312^\010\330v\032\0074\244?\357\227\351\305\023(G\250/w\217\001\240$\346BS\354c"

In [8]:
response = loop.run_until_complete(cli.query_info(
               requestor=org1_admin,
               channel_name='assetchannel',
               peers=['peer0.org1.example.com'],
               decode=True
               ))
response

response status 200


height: 5
currentBlockHash: "\237.\217NQ|)\232\240<\013\250\276g\273\0377\021\030\205b\037\243\203K%\276\r\016\344\341\220"
previousBlockHash: "\215\273\233\312^\010\330v\032\0074\244?\357\227\351\305\023(G\250/w\217\001\240$\346BS\354c"

In [14]:
test_hash = response.currentBlockHash
test_hash

b'\xd3\xf1\xd4\xf2\xect\xf7g\xfc\xc1J\xdcTU(bA\xab]6\xbf)\xf9\xad=:\xe5\x0e\x10\x13HP'

In [15]:
response = loop.run_until_complete(cli.query_block_by_hash(
               requestor=org1_admin,
               channel_name='assetchannel',
               peers=['peer0.org1.example.com'],
               block_hash=test_hash,
               decode=True
               ))
response

response status 200
looking at block 2


{'header': {'number': 2,
  'previous_hash': b'a80d510ea981ced98798e18ef8c850f7ae8c8ac1e91477141f2d263bb0646f97',
  'data_hash': b'b2139b1f7c6b3a682dc7b8166ac9ce4f9086fc67ded82b28b94dd97250a9063e'},
 'data': {'data': [{'signature': b'0E\x02!\x00\xfb\x87=\x99\xc9`\xfc\xe9\xf8\x18>\x1c\xbe\xffi\xce\x105\xc9\xca\xc1oRv\xd7pE\xbb\xb7\x94\xb6\xa0\x02 kW\x82@Z\xaa\xb0t\xad\xf1\xb9L\xc1nA\xfb-\xa2\xfc\xbaS\x07E\xf1R\x8c\x84\x8c\x99\x8a\x83\x93',
    'payload': {'header': {'channel_header': {'type': 3,
       'version': 1,
       'timestamp': '2021-09-12 06:05:11',
       'channel_id': 'assetchannel',
       'tx_id': '5d177d89cdd936d3e22e9ee7d6d6346fb796108287caacd4d25ac894209411d7',
       'epoch': 0,
       'extension': b'\x12\x06\x12\x04atcc',
       'type_string': 'ENDORSER_TRANSACTION'},
      'signature_header': {'creator': {'mspid': 'Org1MSP',
        'id_bytes': '-----BEGIN CERTIFICATE-----\nMIICKjCCAdCgAwIBAgIQEn3uLYlL4sXXQBS1/k8u7zAKBggqhkjOPQQDAjBzMQsw\nCQYDVQQGEwJVUzETMBEGA1UECBMKQ2

In [16]:
tx_id = response.get('data').get('data')[0].get(
    'payload').get('header').get(
    'channel_header').get('tx_id')

tx_id

'5d177d89cdd936d3e22e9ee7d6d6346fb796108287caacd4d25ac894209411d7'

In [17]:
response = loop.run_until_complete(cli.query_block_by_txid(
               requestor=org1_admin,
               channel_name='assetchannel',
               peers=['peer0.org1.example.com'],
               tx_id=tx_id,
               decode=True
               ))

response

response status 200
looking at block 2


{'header': {'number': 2,
  'previous_hash': b'a80d510ea981ced98798e18ef8c850f7ae8c8ac1e91477141f2d263bb0646f97',
  'data_hash': b'b2139b1f7c6b3a682dc7b8166ac9ce4f9086fc67ded82b28b94dd97250a9063e'},
 'data': {'data': [{'signature': b'0E\x02!\x00\xfb\x87=\x99\xc9`\xfc\xe9\xf8\x18>\x1c\xbe\xffi\xce\x105\xc9\xca\xc1oRv\xd7pE\xbb\xb7\x94\xb6\xa0\x02 kW\x82@Z\xaa\xb0t\xad\xf1\xb9L\xc1nA\xfb-\xa2\xfc\xbaS\x07E\xf1R\x8c\x84\x8c\x99\x8a\x83\x93',
    'payload': {'header': {'channel_header': {'type': 3,
       'version': 1,
       'timestamp': '2021-09-12 06:05:11',
       'channel_id': 'assetchannel',
       'tx_id': '5d177d89cdd936d3e22e9ee7d6d6346fb796108287caacd4d25ac894209411d7',
       'epoch': 0,
       'extension': b'\x12\x06\x12\x04atcc',
       'type_string': 'ENDORSER_TRANSACTION'},
      'signature_header': {'creator': {'mspid': 'Org1MSP',
        'id_bytes': '-----BEGIN CERTIFICATE-----\nMIICKjCCAdCgAwIBAgIQEn3uLYlL4sXXQBS1/k8u7zAKBggqhkjOPQQDAjBzMQsw\nCQYDVQQGEwJVUzETMBEGA1UECBMKQ2

In [13]:
response = loop.run_until_complete(cli.query_block(
               requestor=org1_admin,
               channel_name='assetchannel',
               peers=['peer0.org1.example.com'],
               block_number='4',
               decode=True
               ))

response

response status 200
looking at block 4


{'header': {'number': 4,
  'previous_hash': b'8dbb9bca5e08d8761a0734a43fef97e9c5132847a82f778f01a024e64253ec63',
  'data_hash': b'a5d0e834effb60d98a181d4d721079b253ed3769cdf91f4067dc2bfe125e437e'},
 'data': {'data': [{'signature': b'0E\x02!\x00\xecv)m\xd4\x96\xc6|!G\xa2\xceJSr)\xde\x83 \x86-\x89\xbb\xa8|\x1a\xbfrJ\xdc\x1c\x82\x02 :\xff\x7f\xfee\x86VD\xbd\x03\x18\xc0\xa8\xbc6\x92\xb0\x94\x1e\x07\x1f\xc4\x06\xe4R\xbaC\xa1\x0b\xd0\x96\xe4',
    'payload': {'header': {'channel_header': {'type': 3,
       'version': 1,
       'timestamp': '2021-09-12 06:28:54',
       'channel_id': 'assetchannel',
       'tx_id': '20b70410fc270f43e873fd8427d67499b3e1b28aa2b9a61094c8d739bb13f0be',
       'epoch': 0,
       'extension': b'\x12\x06\x12\x04atcc',
       'type_string': 'ENDORSER_TRANSACTION'},
      'signature_header': {'creator': {'mspid': 'Org1MSP',
        'id_bytes': '-----BEGIN CERTIFICATE-----\nMIICKjCCAdCgAwIBAgIQEn3uLYlL4sXXQBS1/k8u7zAKBggqhkjOPQQDAjBzMQsw\nCQYDVQQGEwJVUzETMBEGA1UECBMKQ2

In [16]:
response = loop.run_until_complete(cli.query_block(
               requestor=org1_admin,
               channel_name='assetchannel',
               peers=['peer0.org1.example.com'],
               block_number='3',
               decode=True
               ))

response

response status 200
looking at block 3


{'header': {'number': 3,
  'previous_hash': b'd3f1d4f2ec74f767fcc14adc5455286241ab5d36bf29f9ad3d3ae50e10134850',
  'data_hash': b'cbd1e852b6cbae26865ffdb6c55e0ac42f94ab9ddb3e6ce12b75ae87ecab238a'},
 'data': {'data': [{'signature': b'0D\x02 O8\xd5\xd4\x16@\x01\x94\xa3\xcb0z1\xbf\x9dX\xfe\x00V\xe9\xe6gc\xb6\x93\'i\x80\xc2\xa3>\x06\x02 \x13\x04"\xd4\xf3\xe2\xb1\xed\x03\xa92\xb7E\xb6\xfc\xeb\xa8S\x99}t\x04\x8f\xcaPoF\x9a7s\xb8\x87',
    'payload': {'header': {'channel_header': {'type': 3,
       'version': 1,
       'timestamp': '2021-09-12 06:26:55',
       'channel_id': 'assetchannel',
       'tx_id': '2fcf1b9ee48236f94677da5812895dee397f42e6e096b2e8eb8f82466ee8327b',
       'epoch': 0,
       'extension': b'\x12\x06\x12\x04atcc',
       'type_string': 'ENDORSER_TRANSACTION'},
      'signature_header': {'creator': {'mspid': 'Org1MSP',
        'id_bytes': '-----BEGIN CERTIFICATE-----\nMIICKjCCAdCgAwIBAgIQEn3uLYlL4sXXQBS1/k8u7zAKBggqhkjOPQQDAjBzMQsw\nCQYDVQQGEwJVUzETMBEGA1UECBMKQ2FsaWZvcm5